## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-02-35-02 +0000,bbc,Charlie Kirk's widow pays tribute to husband's...,https://www.bbc.com/news/articles/cz9je8lxge4o...
1,2025-09-13-02-35-01 +0000,startribune,Suspect in Charlie Kirk killing became more po...,https://www.startribune.com/officials-plead-fo...
2,2025-09-13-02-34-58 +0000,nypost,Office Depot facing boycott calls after Michig...,https://nypost.com/2025/09/12/us-news/office-d...
3,2025-09-13-02-28-56 +0000,nypost,Hegseth orders Pentagon officials to track dow...,https://nypost.com/2025/09/12/us-news/hegseth-...
4,2025-09-13-02-28-51 +0000,nyt,Former Judge Sushila Karki Named Nepal’s Leade...,https://www.nytimes.com/2025/09/12/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,kirk,53
0,charlie,47
63,trump,25
46,shooting,16
7,suspect,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
268,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,180
5,2025-09-13-02-26-24 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...,152
34,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,150
22,2025-09-13-00-57-55 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,149
93,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,149


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
268,180,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
189,62,2025-09-12-14-46-32 +0000,nypost,Trump predicts ‘my little Communist’ Zohran Ma...,https://nypost.com/2025/09/12/us-news/trump-pr...
277,53,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
5,52,2025-09-13-02-26-24 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...
285,43,2025-09-12-08-00-00 +0000,cbc,"Israel's attack on Qatar was an escalation, bu...",https://www.cbc.ca/news/world/israel-qatar-inv...
26,38,2025-09-13-00-40-07 +0000,nypost,Charlie Kirk’s wife Erika delivers emotional f...,https://nypost.com/2025/09/12/us-news/charlie-...
175,38,2025-09-12-15-47-11 +0000,nypost,Ex-White House press secretary Karine Jean-Pie...,https://nypost.com/2025/09/12/us-news/ex-white...
43,37,2025-09-12-23-35-31 +0000,nypost,Tyler Robinson may have revealed details about...,https://nypost.com/2025/09/12/us-news/tyler-ro...
180,36,2025-09-12-15-18-57 +0000,startribune,Another shocking act of political violence lea...,https://www.startribune.com/charlie-kirk-minne...
63,34,2025-09-12-22-00-15 +0000,nypost,Utah Gov. Spencer Cox tells people to ‘touch g...,https://nypost.com/2025/09/12/us-news/utah-gov...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
